In [ ]:
%load_ext autoreload
%autoreload 1
%matplotlib inline

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt

# FENICSx stuff
import dolfinx, ufl
from mpi4py import MPI
from petsc4py import PETSc

%aimport mre_pinn

## Helmholtz equation forward problem

Strong form:

$$
\begin{align}
    \mu \nabla^2 \mathbf{u} + \rho \omega^2 \mathbf{u} = 0
\end{align}
$$

Variational form:

$$
\begin{align}
    \int_\Omega \mu \nabla \mathbf{u} \nabla \mathbf{v} dx - \int_\Omega \rho \omega^2 \mathbf{u} \mathbf{v} dx = 0
\end{align}
$$

In [ ]:
np.issubdtype(PETSc.ScalarType, np.complexfloating)

In [ ]:
# define mesh

n_x = 255
n_y = 255

mesh = dolfinx.mesh.create_rectangle(
    comm=MPI.COMM_WORLD,
    points=[[-1, -1], [1, 1]], n=[n_x, n_y],
    cell_type=dolfinx.mesh.CellType.triangle
)
print((n_x + 1) * (n_y + 1))
mesh.geometry.x.shape

In [ ]:
# define elements and function spaces

S = dolfinx.fem.FunctionSpace(mesh, ('Lagrange', 1))
V = dolfinx.fem.VectorFunctionSpace(mesh, ('Lagrange', 1))

In [ ]:
# define boundary condition

def u_bc_func(x):
    n_coords = x.shape[1]
    return np.array([[0.1, -0.1]] * n_coords).T

u_bc = dolfinx.fem.Function(V)
u_bc.interpolate(u_bc_func)

def on_boundary(x):
    return np.isclose(np.linalg.norm(x, np.inf, axis=0), 1)

# identify dofs on the boundary
boundary_dofs = dolfinx.fem.locate_dofs_geometrical(V, on_boundary)

bc = dolfinx.fem.dirichletbc(u_bc, boundary_dofs)

In [ ]:
# define density and angular frequency constants

rho = 1
omega = 4 * np.pi

# define true elasticity function

def mu_func(x, loc=0, radius=0.25, mu_disk=2, mu_back=1):
    '''
    True elasticity function.
    '''
    print(x.shape)
    dist = np.linalg.norm(x - loc, 2, axis=0)
    
    n_coords = x.shape[1]
    value = np.full(n_coords, mu_back)
    value[dist < radius] = mu_disk
    return value

mu = dolfinx.fem.Function(S)
mu.interpolate(mu_func)

In [ ]:
# define variational Helmholtz problem

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
zero = dolfinx.fem.Constant(mesh, np.zeros(2))

# setup variational problem
a = ufl.inner(mu * ufl.grad(u), ufl.grad(v)) * ufl.dx - rho * omega**2 * ufl.inner(u, v) * ufl.dx
L = ufl.inner(zero, v) * ufl.dx

# compute FEM solution
problem = dolfinx.fem.petsc.LinearProblem(a, L, [bc])
u = problem.solve()
print('Done')

In [ ]:
# evaluate functions on mesh

coords = mesh.geometry.x

# identify cells that contain the coordinates
tree = dolfinx.geometry.BoundingBoxTree(mesh, mesh.geometry.dim)
cells = dolfinx.geometry.compute_collisions(tree, coords)
cells = dolfinx.geometry.compute_colliding_cells(mesh, cells, coords)
cells = [cells.links(i)[0] for i in range(coords.shape[0])]

u_vals = u.eval(coords, cells)
mu_vals = mu.eval(coords, cells)

In [ ]:
# visualize the elasticity and displacement

emap = mre_pinn.visual.elast_color_map()
wmap = mre_pinn.visual.wave_color_map()
emin, emax = (0, 10)
wmin, wmax = (-0.25, 0.25)

fig, axes = plt.subplots(1, 3, figsize=(13,4))
axes[0].scatter(coords[:,0], coords[:,1], c=mu_vals, s=1, cmap=emap, vmin=emin, vmax=emax)
axes[1].scatter(coords[:,0], coords[:,1], c=u_vals[:,0], s=1, cmap=wmap, vmin=wmin, vmax=wmax)
axes[2].scatter(coords[:,0], coords[:,1], c=u_vals[:,1], s=1, cmap=wmap, vmin=wmin, vmax=wmax)